# 

# Collect data given a list of college

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
from tqdm import tqdm
import time
import numpy as np
import urllib.request
import csv
import datetime
import itertools
import random
import json
import subprocess

START_DATE = datetime.date(2019, 8, 1)
END_DATE = datetime.date(2019, 12, 1)
FULL_DATASET = False
PATH = 'C:\\Users\\14106\\GAT\\100school\\'

In [7]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)
        
def get_us_subreddits():
    os.chdir(PATH)
    dat = pd.read_csv('colleges.csv')
    schools = dict()
    for i in range(len(dat.name)):
        if dat.loc[i, 'location'].split(', ')[-1] == 'United States':
            schools[dat.loc[i, 'subreddit']] = [dat.loc[i, 'name'], dat.loc[i, 'location']]
    subred = list(schools.keys())
    return subred

def get_subreddits():
    os.chdir(PATH)
    dat = pd.read_csv('colleges.csv')
    schools = []
    for i in range(len(dat.name)):
        schools.append(dat.loc[i, 'subreddit'])
    return schools

def create_school_year_datafile(s):
    os.chdir(PATH)
    att_submission = ["time","author","author_fullname","id","selftext","subreddit","subreddit_subscribers","title","all_awardings","allow_live_comments","can_mod_post","contest_mode","domain","gildings","is_crosspostable","is_meta","is_original_content","is_reddit_media_domain","is_robot_indexable","is_self","is_video","locked","media_only","no_follow","num_comments","num_crossposts","over_18","parent_whitelist_status","pinned","pwls","score","send_replies","spoiler","stickied","thumbnail","total_awards_received","whitelist_status","wls"]
    att_comment = ["time","author","author_fullname","body","gildings","id","link_id","score","subreddit","no_follow","parent_id","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
    y = str(START_DATE)[0:4]
    name = 'comment'+'_'+s+'_'+y+'.csv'
    e = open(name, 'w',newline='')
    with e:
        writer = csv.writer(e)
        writer.writerow(att_comment)
    name = 'submission'+'_'+s+'_'+y+'.csv'
    e = open(name, 'w',newline='')
    with e:
        writer = csv.writer(e)
        writer.writerow(att_submission)

def txt2csv(s, t, y, data):
    '''
    process text to a csv
    s: str - subreddit name
    t: str - comment or submission
    y: str - year
    data: str - a copy of text returned by pushshift endpoint searching
    '''
    att_submission = ["author","author_fullname","id","selftext","subreddit","subreddit_subscribers","title","all_awardings","allow_live_comments","can_mod_post","contest_mode","domain","gildings","is_crosspostable","media_only","no_follow","num_comments","num_crossposts","over_18"]
    att_comment = ["author","author_fullname","body","gildings","id","link_id","score","subreddit","no_follow","parent_id","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
    if t=='comment':
        att = att_comment
        length = len(att_comment)
    elif t=='submission':
        att = att_submission
        length = len(att_submission)
    data=data.replace('true','True')
    data=data.replace('false','False')
    data=data.replace('null','None')
    data='g='+str(data)
    lcls = locals()
    #print(data)
    exec(data, globals(), lcls )
    g = lcls["g"]
    print(g)
    #print(g) 
    for i,val in enumerate(g['data']):
        for j,v in val.items():
            if isinstance(v,dict) or isinstance(v,list):
                val[j]=[str(v)]
            else:
                val[j]=[v]
        if t=='comment':
            values=[val[ii] if ii in val.keys() else 'NA' for ii in att_comment] 
            val={att_comment[ii]:values[ii] for ii in range(len(att_comment))}
        elif t=='submission':
            values=[val[ii] if ii in val.keys() else 'NA' for ii in att_submission] 
            val={att_submission[ii]:values[ii] for ii in range(len(att_submission))}
        df_temp=pd.DataFrame.from_dict(val, orient='columns')
        if i==0:
            df=df_temp
        else:
            df=pd.concat([df,df_temp])
    name = t+'_'+s+'_'+y+'.csv'
    #print(len(g['data']))
    if len(g['data'])>0:
        if os.path.isfile(PATH+name):
            df.to_csv(name, mode='a', index=False, header=False, encoding="utf-8-sig")
        else:
            df.to_csv(name, mode='a', index=False, header=True, encoding="utf-8-sig")
    
def download_single_day(d1, s):
    '''
    download data using pushshift API and save it 
    d1: datetime.date - start date of data
    s: str - subreddit name
    '''
    d2 = d1 + datetime.timedelta(days=1)
    d1_time = datetime.datetime.combine(d1, datetime.time(0,0))
    d2_time = datetime.datetime.combine(d2, datetime.time(0,0))
    os.chdir(PATH)
    url_comment = 'https://api.pushshift.io/reddit/search/comment/?since='+str(int(d1_time.timestamp()))+'&until='+str(int(d2_time.timestamp()))+'&subreddit='+s+'&size=1000'
    url_submission = 'https://api.pushshift.io/reddit/search/submission/?since='+str(int(d1_time.timestamp()))+'&until='+str(int(d2_time.timestamp()))+'&subreddit='+s+'&size=1000'
    req = requests.get(url_comment)
    time.sleep(3)
    content = req.text
    txt2csv(s=s, t='comment', y=str(d1)[0:4],data=content)
    #req = requests.get(url_submission)
    #time.sleep(3)
    #content = req.text
    #txt2csv(s=s, t='submission', y=str(d1)[0:4],data=content)

In [10]:

'''
subred=get_subreddits()
for s in tqdm(subred):
    create_school_year_datafile(s)
    download_single_day(d1=START_DATE, d2=END_DATE, s=s)

'''
#download day by day
if FULL_DATASET:
    subred=get_subreddits()
else:
    subred=["columbia"]
    #subred=["berkeley","Harvard","notredame","dartmouth","uCinci","Athens","Chicago","reedcollege","usna"]
    #subred=["uCinci","Athens","Chicago","reedcollege","usna"]
#subred=subred[109:]
for s in tqdm(subred[0:]):
    #create_school_year_datafile(s)
    for single_date in daterange(START_DATE, END_DATE):
        try:
            download_single_day(d1=single_date, s=s)
        except:
            download_single_day(d1=single_date, s=s)



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

{'detail': 'Not authenticated'}


  0%|                                                                                            | 0/1 [00:06<?, ?it/s]

{'detail': 'Not authenticated'}


KeyError: 'data'

In [7]:
subred=get_subreddits()
subred.index('UniversityofVermont')

119

In [37]:
#check if all subreddites are in 
subred=get_subreddits()
y='2020'
t='comment'
files=set(os.listdir(PATH))
empty_school=[]
for y in ['2021', '2022']:
    for s in subred:
        name = t+'_'+s+'_'+y+'.csv'
        if name not in files:
            empty_school.append(name)
print(empty_school)

['comment_ProvidenceCollege_2021.csv', 'comment_UMT_2021.csv', 'comment_UniversityOfToledo_2021.csv', 'comment_uwyo_2021.csv', 'comment_SHU_2022.csv', 'comment_UMT_2022.csv', 'comment_UniversityOfToledo_2022.csv', 'comment_uwyo_2022.csv', 'comment_WellesleyCollege_2022.csv']


In [77]:
os.chdir('C:\\Users\\14106\\GAT\\100school\\columbia\\')
df=pd.read_csv('comment_columbia_2022.csv',skip_blank_lines=True)#, delimiter=' ', escapechar='\\',encoding='utf-8')
os.chdir('C:\\Users\\14106\\GAT\\100school\\')
#for i in range(len(df.index)):
#    eg=data.body[i]
#    df.loc[i,'body']=eg.encode('utf-8').decode('unicode-escape', errors='surrogatepass')
df.to_csv('comment_columbia_2022.csv', index=False, header=True, encoding="utf-8-sig")


In [75]:
import unidecode
unidecode.unidecode('I don\u2019t think GS students are allowed to only have a concentration. I think I read somewhere that every GSer must have a major.')
unidecode.unidecode(data.body[4])
type(data.body[4])
eg=data.body[4]
eg.encode('utf-8').decode('unicode-escape', errors='surrogatepass')

os.chdir('C:\\Users\\14106\\GAT\\100school\\columbia\\')
#df=pd.read_csv('columbia2019.csv',skip_blank_lines=True)#, delimiter=' ', escapechar='\\',encoding='utf-8')
df.body[4]#.encode('utf-8')#.decode('unicode-escape', errors='surrogatepass')

os.chdir('C:\\Users\\14106\\GAT\\100school\\')
df=pd.read_csv('comment_columbia_2019.csv',skip_blank_lines=True,encoding='utf-8')#, delimiter=' ', escapechar='\\',encoding='utf-8')
#df.body[4].encode('utf-8').decode('unicode-escape')

In [14]:
os.chdir(PATH)
data=content
att_comment = ["author","author_fullname","body","gildings","id","link_id","score","subreddit","no_follow","parent_id","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att = att_comment
length = len(att_comment)
data=data.replace('true','True')
data=data.replace('false','False')
data=data.replace('null','None')
data='g='+str(data)
lcls = locals()
#print(data)
exec(data, globals(), lcls )
g = lcls["g"]
#print(g)
for i,val in enumerate(g['data']):
    for j,v in val.items():
        if isinstance(v,dict) or isinstance(v,list):
            val[j]=[str(v)]
        else:
            val[j]=[v]
    values=[val[ii] if ii in val.keys() else 'NA' for ii in att_comment] 
    val={att_comment[ii]:values[ii] for ii in range(len(att_comment))}
    df_temp=pd.DataFrame.from_dict(val, orient='columns')
    if i==0:
        df=df_temp
    else:
        df=pd.concat([df,df_temp])
df.to_csv('trial.csv', mode='a', index=False, header=True, encoding="utf-8-sig")

In [23]:
#https://api.pushshift.io/reddit/search/comment/?since=1633060800&until=1665115200&subreddit=notredame&size=1000
import datetime as dt

since = int(dt.datetime(2021,10,1,0,0).timestamp())
until = int(dt.datetime(2022,10,7,0,0).timestamp())
print(since, until)

1633060800 1665115200


In [106]:
import requests
from datetime import datetime
import traceback
import time
import json
import sys
import csv
import json

username = ""  # put the username you want to download in the quotes
subreddit = "notredame"  # put the subreddit you want to download in the quotes
thread_id = ""  # put the id of the thread you want to download in the quotes, it's the first 5 to 7 character string of letters and numbers from the url, like 107xayi
# leave either one blank to download an entire user's or subreddit's history
# or fill in both to download a specific users history from a specific subreddit

# change this to one of "human", "csv" or "json"
# - human: the score, creation date, author, link and then the comment/submission body on a second line. Objects are separated by lines of dashes
# - csv: a comma seperated value file with the fields score, date, title, author, link and then body or url
# - json: the full json object
output_format = "csv"

# default start time is the current time and default end time is all history
# you can change out the below lines to set a custom start and end date. The script works backwards, so the end date has to be before the start date
start_time = datetime.strptime("09/25/2021", "%m/%d/%Y")  #datetime.utcnow()  #datetime.strptime("10/05/2021", "%m/%d/%Y")
end_time = datetime.strptime("12/1/2021", "%m/%d/%Y")  #datetime.strptime("09/25/2021", "%m/%d/%Y")

convert_to_ascii = False  # don't touch this unless you know what you're doing
convert_thread_id_to_base_ten = True  # don't touch this unless you know what you're doing

    
def write_human_line(handle, obj, is_submission, convert_to_ascii):
    handle.write(str(obj['score']))
    handle.write(" : ")
    handle.write(datetime.fromtimestamp(obj['created_utc']).strftime("%Y-%m-%d"))
    if is_submission:
        handle.write(" : ")
        if convert_to_ascii:
            handle.write(obj['title'].encode(encoding='ascii', errors='ignore').decode())
        else:
            handle.write(obj['title'])
    handle.write(" : u/")
    handle.write(obj['author'])
    handle.write(" : ")
    handle.write(f"https://www.reddit.com{obj['permalink']}")
    handle.write("\n")
    if is_submission:
        if obj['is_self']:
            if 'selftext' in obj:
                if convert_to_ascii:
                    handle.write(obj['selftext'].encode(encoding='ascii', errors='ignore').decode())
                else:
                    handle.write(obj['selftext'])
        else:
            handle.write(obj['url'])
    else:
        if convert_to_ascii:
            handle.write(obj['body'].encode(encoding='ascii', errors='ignore').decode())
        else:
            handle.write(obj['body'])
    handle.write("\n-------------------------------\n")


def write_csv_line(writer, obj, is_submission):
    output_list = []
    print(type(obj))
    output_list.append(str(obj['score']))
    output_list.append(datetime.fromtimestamp(obj['created_utc']).strftime("%Y-%m-%d"))
    if is_submission:
        output_list.append(obj['title'])
    output_list.append(f"u/{obj['author']}")
    output_list.append(f"https://www.reddit.com{obj['permalink']}")
    if is_submission:
        if obj['is_self']:
            if 'selftext' in obj:
                output_list.append(obj['selftext'])
            else:
                output_list.append("")
        else:
            output_list.append(obj['url'])
    else:
        output_list.append(obj['body'])
    writer.writerow(output_list)


def write_json_line(handle, obj):
    handle.write(json.dumps(obj))
    handle.write("\n")


def download_from_url(filename, url_base, output_format, start_datetime, end_datetime, is_submission, convert_to_ascii):
    print(f"Saving to {filename}")

    count = 0
    if output_format == "human" or output_format == "json":
        if convert_to_ascii:
            handle = open(filename, 'w', encoding='ascii')
        else:
            handle = open(filename, 'w', encoding='UTF-8')
    else:
        handle = open(filename, 'w', encoding='UTF-8', newline='')
        writer = csv.writer(handle)

    previous_epoch = int(start_datetime.timestamp())
    break_out = False
    while True:
        new_url = url_base+str(previous_epoch)
        json_text = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
        time.sleep(1)  # pushshift has a rate limit, if we send requests too fast it will start returning error messages
        try:
            json_data = json_text.json()
        except json.decoder.JSONDecodeError:
            time.sleep(1)
            continue

        if 'data' not in json_data:
            break
        objects = json_data['data']
        if len(objects) == 0:
            break

        for obj in objects:
            previous_epoch = obj['created_utc'] - 1
            if end_datetime is not None and datetime.utcfromtimestamp(previous_epoch) < end_datetime:
                break_out = True
                break
            count += 1
            try:
                if output_format == "human":
                    write_human_line(handle, obj, is_submission, convert_to_ascii)
                elif output_format == "csv":
                    write_csv_line(writer, obj, is_submission)
                elif output_format == "json":
                    write_json_line(handle, obj)
            except Exception as err:
                if 'permalink' in obj:
                    print(f"Couldn't print object: https://www.reddit.com{obj['permalink']}")
                else:
                    print(f"Couldn't print object, missing permalink: {obj['id']}")
                print(err)
                print(traceback.format_exc())

        if break_out:
            break

        print(f"Saved {count} through {datetime.fromtimestamp(previous_epoch).strftime('%Y-%m-%d')}")

    print(f"Saved {count}")
    handle.close()


if __name__ == "__main__":
    filter_string = None
    if username == "" and subreddit == "" and thread_id == "":
        print("Fill in username, subreddit or thread id")
        sys.exit(0)
    if output_format not in ("human", "csv", "json"):
        print("Output format must be one of human, csv, json")
        sys.exit(0)

    filters = []
    if username:
        filters.append(f"author={username}")
    if subreddit:
        filters.append(f"subreddit={subreddit}")
    if thread_id:
        if convert_thread_id_to_base_ten:
            filters.append(f"link_id={int(thread_id, 36)}")
        else:
            filters.append(f"link_id=t3_{thread_id}")
    filter_string = '&'.join(filters)

    url_template = "https://api.pushshift.io/reddit/{}/search?limit=1000&order=desc&{}&before="

    if not thread_id:
        download_from_url("posts.txt", url_template.format("submission", filter_string), output_format, start_time, end_time, True, convert_to_ascii)
    download_from_url("comments.txt", url_template.format("comment", filter_string), output_format, start_time, end_time, False, convert_to_ascii)

Saving to posts.txt
Saved 0
Saving to comments.txt
Saved 0


In [79]:

#val1={ii:val[ii] if ii in val.keys() else ii:None for ii in att_comment}

In [91]:
s

'ucla'

In [22]:
for i,j in val.items():
    print(type(j),j)

<class 'str'> []
<class 'bool'> False
<class 'NoneType'> None
<class 'str'> FlyingA10Brrrt
<class 'int'> 1471296504
<class 'str'> transparent
<class 'NoneType'> None
<class 'str'> [{'e': 'text', 't': 'Carroll'}]
<class 'str'> a5ba18c0-026c-11e8-a2cd-0ef7c9d0a440
<class 'str'> Carroll
<class 'str'> dark
<class 'str'> richtext
<class 'str'> t2_10jose
<class 'bool'> False
<class 'bool'> False
<class 'str'> For thermodynamics, an extra credit assignment was to figure out how long it would take to properly bake a potato then do it and see if it tasted good. For Writing and Rhetoric, I had to go work on three different farms around the South Bend area.
<class 'bool'> True
<class 'bool'> False
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'NoneType'> None
<class 'int'> 0
<class 'int'> 1627844191
<class 'NoneType'> None
<class 'bool'> False
<class 'int'> 0
<class 'str'> {}
<class 'str'> h7cczxy
<class 'bool'> False
<class 'str'> t3_ovlsjq
<class 'bool'> False
<

# collect features for selected schools

In [ ]:
def find_region(state):
    region={('Northeast','New England'):['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont'], 
            ('Northeast','Middle Atlantic'):['New Jersey', 'New York', 'Pennsylvania'],
            ('Midwest','East North Central'):['Indiana', 'Illinois', 'Michigan', 'Ohio', 'Wisconsin'],
            ('Midwest','West North Central'):['Iowa', 'Nebraska', 'Kansas', 'North Dakota', 'Minnesota', 'South Dakota', 'Missouri'],
            ('South','South Atlantic'):['Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'West Virginia'],
            ('South','East South Central'):['Alabama', 'Kentucky', 'Mississippi', 'Tennessee'],
            ('South','West South Central'):['Arkansas', 'Louisiana', 'Oklahoma', 'Texas'],
            ('West','Mountain'):['Arizona', 'Colorado', 'Idaho', 'New Mexico', 'Montana', 'Utah', 'Nevada', 'Wyoming'],
            ('West','Pacific'):['Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']}
    for i in region.keys():
        if state in region[i]:
            return i
    return 'Wrong state.'
    

# draft below

In [ ]:
import numpy as np
import urllib.request  
import os
import csv
import requests
import time
#schools = ["notredame","dartmouth"]
#schools=["uofm"]
#schools=["columbia"]
#schools=["UCSD","ucla"]
#schools=["berkeley"]
schools=["Harvard"]
months=['08','09','10','11','12']
types=["comment","submission"]
years=['2019','2020']
att_submission=["time","author","author_fullname","id","selftext","subreddit","subreddit_subscribers","title","all_awardings","allow_live_comments","can_mod_post","contest_mode","domain","gildings","is_crosspostable","is_meta","is_original_content","is_reddit_media_domain","is_robot_indexable","is_self","is_video","locked","media_only","no_follow","num_comments","num_crossposts","over_18","parent_whitelist_status","pinned","pwls","score","send_replies","spoiler","stickied","thumbnail","total_awards_received","whitelist_status","wls"]
att_comment=["time","author","author_fullname","body","gildings","id","link_id","score","subreddit","no_follow","parent_id","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att_author=[]

t='comment'
#dat=att_comment
att=att_comment
length=len(att_comment)

In [ ]:

f='yale_comment.txt'
name='example.csv'
e=open(name, 'w',newline='')
with e:
    writer = csv.writer(e)
    writer.writerow(att)

In [ ]:



def txt2csv(f, s, t, y, time):
    '''
    process text to a csv
    f: str - filename
    s: str - subreddit name
    t: str - comment or submission
    y: str - year
    time: str - date
    '''
    att_submission=["time","author","author_fullname","id","selftext","subreddit","subreddit_subscribers","title","all_awardings","allow_live_comments","can_mod_post","contest_mode","domain","gildings","is_crosspostable","is_meta","is_original_content","is_reddit_media_domain","is_robot_indexable","is_self","is_video","locked","media_only","no_follow","num_comments","num_crossposts","over_18","parent_whitelist_status","pinned","pwls","score","send_replies","spoiler","stickied","thumbnail","total_awards_received","whitelist_status","wls"]
    att_comment=["time","author","author_fullname","body","gildings","id","link_id","score","subreddit","no_follow","parent_id","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
    if t=='comment':
        att=att_comment
        length=len(att_comment)
    elif t=='submission':
        att=att_submission
        length=len(att_submission)
    dat=np.empty((0,length), str)
    q=open(f, 'r')
    Lines = q.readlines() 
    oneline=["" for x in range(len(att))]
    oneline[0]=time
    for line in Lines[2:(len(Lines)-2)]:
        for a in range(len(att)):
            if len(line.split('"'))>1 and line.split('"')[1]==att[a]:
                oneline[a]=''.join(line.split(': ')[1:])[0:-2]
            elif len(list(line.split('}')[0]))<9 and list(line.split(' ')[-1])[0]=='}':
                dat=np.vstack ((dat, oneline))
                oneline=["" for x in range(len(att))]
                oneline[0]=time
                break
    name = t+s+y+'.csv'
    e=open(name, 'a',newline='')
    with e:
        writer = csv.writer(e)
        for row in dat:
            writer.writerow(row)

In [ ]:
tuple((4,2))